# Ahmad M. Osman - Dr. Lee, DS320


## Assignment Instructions
In this exam, there are two main parts to turn in. You will turn both in as one Jupyter Notebook. 

A Python program that when run, builds a CSV file of the top 500 movies with all the data in the IMDB database plus the Adjusted Gross box office.
A record of your work from following chapter 3 of the textbook. You must include comments as to what you are doing at each step and what you find as you are working through the chapter. 
Please read below for more details on both parts of this Jupyter Notebook.

For the first part you are to write a computer program that builds your CSV file from the sources of information listed below. You should define a function called "main" that when called would create the file. All your code should be in the main function or should be called from the main function (you may define other functions as well if you like). 

You should comment this code so it is clear what you are doing. Before putting this code in your Jupyter Notebook, it will probably be best to write the program in Wing IDE 101 or PyCharm. That way you can use the debugger while running it. 

In this program you are to compute the top grossing movies of all time from the 500 best movies. Use the IMDB database (the API to gather information) and the CPI inflation adjustment calculator. Adjust all box office amounts into August 2018 dollars. 

Be careful to write the program to do this for five movies first, then expand it to the top 500 when you have figured out that you have all the right information. Build a CSV file of your results. Put this CSV file in your knuth public_html folder for yourself. 

To complete this project/exam you will need to provide a Jupyter Notebook with your code in it to build this CSV file. However, it may be better to write this code using Wing or PyCharm to begin with. Here are the steps that are involved in building this CSV.

Go to the 500 best movies page and get the list of 500 movie IDs from the page. This will come in the form of a JSON file and you will have a dictionary in your program of the 500 movies.
Open a CSV file to write your results to.
Iterate through this list of movies and use each Id in the list to go to the OMDB API to get each movie's information. Again this will come back as JSON data so you will end up with a dictionary in your program.
For each movie's JSON data you should see if the value of the box office is a valid float. If it is, then use the CPI calculator API to get the value of $1 in the year of the movie in August of 2018 dollars. Multiply the Box Office of the movie times this factor to get the Adjusted Gross box office value. Add this new column of data to your dictionary of the movie information. If the Box Office is not available, then still add the Adjusted Gross as 0.
Delete the Ratings column that has the sources of all the ratings. They are duplicated from other columns anyway so they can be deleted from the dictionary.
Write the dictionary values to a file in comma separated form. If this is the first row to be written to the file, write the keys of the dictionary first. Otherwise, just write the values to the file. The keys are the column headers and should appear as the first row in your CSV file. 
Close the file when you are all done. 
After creating this file, give it a unique name and upload it to your public_html directory on knuth so you (and I) can access it. Give it a unique name that you don't share with others but use the name in your Python notebook. Copy all the code from your working program into the Python Notebook so I can see it along with the rest of your work as you work through Chapter 3 of the Pandas Textbook. 

You can leave out the call to your main function in your Python Notebook so the code to build the CSV never actually executes once it is in the notebook.

You should get the program to work with five movies first before you expand this to all 500 best movies of all time because you are limited to 1000 calls to the IMDB database in a day.

For the second part, read chapter 3 of the Pandas Cookbook and apply what you read there to this problem. You may skip the final "Calculating a trailing stop order price" recipe on page 111 of the textbook.

Use Pandas to explore this data using what you read in Chapter 3. Please store your CSV in your public_html director and reference it with a fully qualified name from your Python Notebook. Upload your Python notebook here with your comments as to what you are looking at in the data set. 

This assignment/exam requires you to do some screen scraping and to read a JSON file. Your goal is to build a CSV file with the data from the IMDB web api in it (as described above) for the top 500 movies with an "Adjusted Box Office" amount adjusted to August 2018 dollars. 

You should turn in your Python Notebook here having worked through the Chapter 3 material. You should find at least one interesting thing in the data through your data analysis. You must comment on that interesting thing in Python Notebook and show how you found it. 

When you turn in your notebook, make sure it directly references your data on knuth and do not share the name of your CSV file with others in the class. You must access your own CSV file in this exam question but you do not need to upload the CSV file here.

# Interesting Things I Found After Finishing the Assignment


* Christopher Nolan is the highest frequent director for the top 500 movies!
* There are 2 beauty and the beast movies
* The Godfather is the highest rated movie with 9.2 imdbRating and a 100 for the Metascore.
* Metascore is the rating of critics while IMDB rating is for users on the website. 
* There is a correlation of 0.42, which is moderate, between Metascore and IMDB rating.
* The older the movie, the higher the rating, can be seen as there is a negative correlation between Metascore/imdbRating and Year. This is stronger for critics than IMDB users.


# Downloading Data

In [253]:
import requests
from lxml import html
import json
import csv

def main():
    # Using XPath and JSON to extract a dictionary of all top 500 IMDB movies IDs 
    imdb = requests.get("https://www.imdb.com/list/ls003073623/")
    tree = (html.fromstring(imdb.content))
    movies = tree.xpath('//html/head/script[@type="application/ld+json"]/text()')[0]
    
    # We have our dictionary now
    jsonDict = json.loads(movies)

    # Lets get the urls that contain the ids
    jsonDict = jsonDict['about']['itemListElement']
    # Now we will have a list of IDs
    imdbIDs = [id['url'].split('/')[2] for id in jsonDict]

    # Preparing CSV file for storage
    headers_written = False
    filename = "osmanTop500MoviesInflationAdjusted.csv"

    # A context window to write each movie to the CSV file
    with open (filename, 'a') as csvfile:
        # Looping through the movies IDs, getting the info we want, doing data cleaning and finding the adjusted box office value, and then we save to the CSV file
        for imdbID in imdbIDs:
            # Now we use OMDB to get all the information we want
            movie = requests.get("http://www.omdbapi.com/?apikey=47734a01&i=" + imdbID)
            movie = json.loads(movie.content)
            
            # We clean the data - and get adjusted box office value            
            try:
                movie["imdbVotes"] = str(movie["imdbVotes"]).replace(',', '')
                movie["Released"] = str(movie["Released"]).replace(' ', '-')
                movie["Month"] = str(movie["Released"].split('-')[1])
                movie["DVD"] = str(movie["DVD"]).replace(' ', '-')
            except:
                pass

            # Preparing for inflation value factor
            # https://data.bls.gov/cgi-bin/cpicalc.pl?cost1=1&year1=199905&year2=201808
            months = { 'Jan' : '01', 'Feb' : '02', 'Mar' : '03', 'Apr' : '04', 'May' : '05', 'Jun' : '06', 'Jul' : '07', 'Aug' : '08', 'Sep' : '09', 'Oct' : '10', 'Nov' : '11', 'Dec' : '12' }
            
            try:
                # Cleaning BoxOffice values and Calculating the Adjusted Box Office Value
                movie["BoxOffice"] = str(movie["BoxOffice"]).replace(',', '')
                movie["BoxOffice"] = str(movie["BoxOffice"]).replace('$', '')

                # Getting adjusted value factor and calculating the adjusted box office value
                cpiStr = "https://data.bls.gov/cgi-bin/cpicalc.pl?cost1=1&year1=" + str(movie["Year"]) + str(months[movie["Month"]]) + "&year2=201808"
                cpiPage = requests.get(cpiStr)
                cpiTree = html.fromstring(cpiPage.content)
                factor = float(cpiTree.xpath('//span[@id="answer"]/text()')[0][1:].replace(',',''))
                movie["AdjustedForInflationBoxOffice"] = float(movie["BoxOffice"]) * factor
            except:
                movie["AdjustedForInflationBoxOffice"] = 0

            # Deleing the Ratings data - we do not need it
            del movie["Ratings"]

            # Starting to write to CSV file for this loop entry
            headers = movie.keys()
            writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n',fieldnames=headers)

            # Writing CSV file header once - at the first iteration of the loop
            if not headers_written:
                writer.writeheader()
                headers_written = True

            # Writing movie data to the CSV file
            writer.writerow(movie)


# if __name__ == "__main__":
    # main()


# Chapter 3: Beginning Data Analysis

## Recipes
* [Developing a data analysis routine](#Developing-a-data-analysis-routine)
* [Selecting the smallest of the largest](#Selecting-the-smallest-of-the-largest)
* [Selecting the largest of each group by sorting](#Selecting-the-largest-of-each-group-by-sorting)
* [Replicating nlargest with sort_values](#Replicating-nlargest-with-sort_values)
* [Correlations](#Correlations)
* [Reducing memory by changing data types](#Reducing-memory-by-changing-data-types)

In [254]:
# we need these libraries
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = 50

# Developing a data analysis routine

In [255]:
# reading the gather info from knuth
movies = pd.read_csv('http://knuth.luther.edu/~osmaah02/osmanTop500MoviesInflationAdjusted.csv')

In [256]:
# looking at the first five rows
movies.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Month,AdjustedForInflationBoxOffice
0,The Godfather,1972,R,24-Mar-72,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay by), Francis Ford Coppo...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 24 wins & 28 nominations.,https://m.media-amazon.com/images/M/MV5BM2MyNj...,100.0,9.2,1370696.0,tt0068646,movie,9-Oct-01,NaN,Paramount Pictures,http://www.thegodfather.com,True,Mar,0.0
1,Raiders of the Lost Ark,1981,PG,12-Jun-81,115 min,"Action, Adventure",Steven Spielberg,"Lawrence Kasdan (screenplay by), George Lucas ...","Harrison Ford, Karen Allen, Paul Freeman, Rona...","In 1936, archaeologist and adventurer Indiana ...","English, German, Hebrew, Spanish, Arabic, Nepali",USA,Won 4 Oscars. Another 30 wins & 23 nominations.,https://m.media-amazon.com/images/M/MV5BMjA0OD...,85.0,8.5,776796.0,tt0082971,movie,21-Oct-03,NaN,Paramount Pictures,http://www.indianajones.com/raiders/,True,Jun,0.0
2,Star Wars: Episode V - The Empire Strikes Back,1980,PG,20-Jun-80,124 min,"Action, Adventure, Fantasy, Sci-Fi",Irvin Kershner,"Leigh Brackett (screenplay by), Lawrence Kasda...","Mark Hamill, Harrison Ford, Carrie Fisher, Bil...",After the rebels are brutally overpowered by t...,English,USA,Won 1 Oscar. Another 21 wins & 19 nominations.,https://m.media-amazon.com/images/M/MV5BYmU1ND...,82.0,8.8,1007473.0,tt0080684,movie,21-Sep-04,4548170.0,Twentieth Century Fox,http://www.starwars.com/episode-v/,True,Jun,13871918.5
3,The Shawshank Redemption,1994,R,14-Oct-94,142 min,Drama,Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 32 n...,https://m.media-amazon.com/images/M/MV5BMDFkYT...,80.0,9.3,2005476.0,tt0111161,movie,27-Jan-98,NaN,Columbia Pictures,NaN,True,Oct,0.0
4,Jaws,1975,PG,20-Jun-75,124 min,"Adventure, Drama, Thriller",Steven Spielberg,"Peter Benchley (screenplay), Carl Gottlieb (sc...","Roy Scheider, Robert Shaw, Richard Dreyfuss, L...","A local sheriff, a marine biologist and an old...",English,USA,Won 3 Oscars. Another 11 wins & 18 nominations.,https://m.media-amazon.com/images/M/MV5BMmVmOD...,87.0,8.0,489473.0,tt0073195,movie,11-Jul-00,NaN,Universal Pictures,http://www.jaws25.com/,True,Jun,0.0


In [257]:
# so we have 500 rows of 26 columns
movies.shape

(500, 26)

In [258]:
# lets look at data description, numerical value variables only
with pd.option_context('display.max_rows', 8):
    display(movies.describe(include=[np.number]).T)

,count,mean,std,min,25%,50%,75%,max
Year,500.0,1.980032e+03,2.077957e+01,1924.0,1966.0,1984.0,1998.0,2008.0
Metascore,379.0,7.920844e+01,1.240184e+01,41.0,70.0,80.0,89.0,100.0
imdbRating,500.0,7.871800e+00,4.879148e-01,5.7,7.6,7.9,8.2,9.3
imdbVotes,499.0,2.706051e+05,3.250212e+05,1839.0,39425.5,152514.0,372048.0,2005476.0
BoxOffice,103.0,9.913201e+07,1.267996e+08,17599.0,5166076.0,44566004.0,155194037.0,533316061.0
AdjustedForInflationBoxOffice,500.0,2.744930e+07,9.370915e+07,0.0,0.0,0.0,0.0,655120000.0


In [259]:
# now lets do the same for categorical variables
movies.describe(include=[np.object, pd.Categorical]).T
# looks like we have two beauty and the beast movies
# also, 2004 seems to be a full of creativity year!

,count,unique,top,freq
Title,500,498,Beauty and the Beast,2
Rated,492,12,R,210
Released,499,483,21-Sep-01,2
Runtime,500,125,110 min,19
Genre,500,179,Drama,37
Director,499,299,Steven Spielberg,11
Writer,497,472,Woody Allen,4
Actors,499,498,"Mark Hamill, Harrison Ford, Carrie Fisher, Bil...",2
Plot,500,500,"A Phoenix secretary embezzles $40,000 from her...",1
Language,499,154,English,210


In [260]:
(movies["Title"] == "Beauty and the Beast").value_counts()

False    498
True       2
Name: Title, dtype: int64

In [261]:
# lets get some info about this DataFrame - 
# This tells us how many non-null objects for each column
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 26 columns):
Title                            500 non-null object
Year                             500 non-null int64
Rated                            492 non-null object
Released                         499 non-null object
Runtime                          500 non-null object
Genre                            500 non-null object
Director                         499 non-null object
Writer                           497 non-null object
Actors                           499 non-null object
Plot                             500 non-null object
Language                         499 non-null object
Country                          500 non-null object
Awards                           483 non-null object
Poster                           500 non-null object
Metascore                        379 non-null float64
imdbRating                       500 non-null float64
imdbVotes                        499 non-null 

In [262]:
# lets count the nulls?
movies.isnull().sum()


Title                              0
Year                               0
Rated                              8
Released                           1
Runtime                            0
Genre                              0
Director                           1
Writer                             3
Actors                             1
Plot                               0
Language                           1
Country                            0
Awards                            17
Poster                             0
Metascore                        121
imdbRating                         0
imdbVotes                          1
imdbID                             0
Type                               0
DVD                               10
BoxOffice                        397
Production                         9
Website                          310
Response                           1
Month                              1
AdjustedForInflationBoxOffice      0
dtype: int64

In [263]:
# looks like we have a lot of them - lets drop the ones that 
# do not have box office values
movies = movies.dropna(subset=["BoxOffice"])

In [264]:
# and we are left with?
movies.count()

Title                            103
Year                             103
Rated                            103
Released                         103
Runtime                          103
Genre                            103
Director                         103
Writer                           103
Actors                           103
Plot                             103
Language                         103
Country                          103
Awards                           103
Poster                           103
Metascore                         99
imdbRating                       103
imdbVotes                        103
imdbID                           103
Type                             103
DVD                              103
BoxOffice                        103
Production                       103
Website                           96
Response                         103
Month                            103
AdjustedForInflationBoxOffice    103
dtype: int64

In [265]:
# one more statistical description on numerical columns
movies.describe(include=[np.number]).T

,count,mean,std,min,25%,50%,75%,max
Year,103.0,1.998417e+03,1.586000e+01,1925.0,2000.50,2004.0,2006.0,2008.0
Metascore,99.0,7.753535e+01,1.197303e+01,46.0,68.00,80.0,85.5,100.0
imdbRating,103.0,7.828155e+00,5.645435e-01,6.1,7.50,7.9,8.2,9.0
imdbVotes,103.0,4.686225e+05,3.679558e+05,6831.0,216119.50,391323.0,632194.5,1969949.0
BoxOffice,103.0,9.913201e+07,1.267996e+08,17599.0,5166076.00,44566004.0,155194037.0,533316061.0
AdjustedForInflationBoxOffice,103.0,1.332490e+08,1.694818e+08,52414.6,8749140.73,62860656.6,190408024.0,655120000.0


In [266]:
# one more statistical description on categorical columns
movies.describe(include=[np.object, pd.Categorical]).T
# I am so happy seeing Christopher Nolan as the highest frequent
# director!

,count,unique,top,freq
Title,103,103,Battleship Potemkin,1
Rated,103,6,R,47
Released,103,99,30-Sep-05,2
Runtime,103,64,102 min,5
Genre,103,65,Drama,6
Director,103,86,Peter Jackson,4
Writer,103,102,"J.R.R. Tolkien (novel), Fran Walsh (screenplay...",2
Actors,103,103,"Matt Damon, Franka Potente, Brian Cox, Julia S...",1
Plot,103,103,A family determined to get their young daughte...,1
Language,103,47,English,34


## There's more...

In [267]:
# lets look at some percentiles, shall we?
with pd.option_context('display.max_rows', 5):
    display(movies.describe(include=[np.number], 
                 percentiles=[.01, .05, .10, .25, .5, .75, .9, .95, .99]).T)

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Year,103.0,1.998417e+03,1.586000e+01,1925.0,1931.1400,1966.20,1985.40,2000.50,2004.0,2006.0,2.007000e+03,2.007000e+03,2.008000e+03,2008.0
Metascore,99.0,7.753535e+01,1.197303e+01,46.0,50.9000,54.80,62.00,68.00,80.0,85.5,9.220000e+01,9.500000e+01,9.804000e+01,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BoxOffice,103.0,9.913201e+07,1.267996e+08,17599.0,26978.4800,146021.00,641454.60,5166076.00,44566004.0,155194037.0,3.180300e+08,3.724401e+08,4.308407e+08,533316061.0
AdjustedForInflationBoxOffice,103.0,1.332490e+08,1.694818e+08,52414.6,295950.6196,539814.12,1779524.04,8749140.73,62860656.6,190408024.0,4.118577e+08,5.190607e+08,6.123510e+08,655120000.0


# Selecting the smallest of the largest

In [268]:
movies.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD',
       'BoxOffice', 'Production', 'Website', 'Response', 'Month',
       'AdjustedForInflationBoxOffice'],
      dtype='object')

In [269]:
# lets look at the least successful movies
movies2 = movies[['Title', 'AdjustedForInflationBoxOffice', 'imdbRating', 'Metascore']]
movies2.head()

,Title,AdjustedForInflationBoxOffice,imdbRating,Metascore
2,Star Wars: Episode V - The Empire Strikes Back,13871918.5,8.8,82.0
14,The Dark Knight,613313470.2,9.0,84.0
15,2001: A Space Odyssey,991382.2,8.3,82.0
22,Back to the Future,6846559.2,8.5,87.0
23,The Lord of the Rings: The Fellowship of the Ring,449020000.0,8.8,92.0


In [270]:
# selecting the top 20 movies by imdbRating
movies2.nlargest(20, 'imdbRating').head()

,Title,AdjustedForInflationBoxOffice,imdbRating,Metascore
14,The Dark Knight,613313470.2,9.0,84.0
33,The Lord of the Rings: The Return of the King,498680000.0,8.9,94.0
2,Star Wars: Episode V - The Empire Strikes Back,13871918.5,8.8,82.0
23,The Lord of the Rings: The Fellowship of the Ring,449020000.0,8.8,92.0
239,Forrest Gump,561000000.0,8.8,82.0


In [271]:
# now lets get the top 20 by imdbRating that are of the smallest 5 of
# AdjustedForInflationBoxOffice
movies2.nlargest(20, 'imdbRating').nsmallest(5, 'AdjustedForInflationBoxOffice')

,Title,AdjustedForInflationBoxOffice,imdbRating,Metascore
211,M,293903.30,8.4,NaN
238,Cinema Paradiso,519997.10,8.5,80.0
63,Oldboy,873755.86,8.4,74.0
15,2001: A Space Odyssey,991382.20,8.3,82.0
22,Back to the Future,6846559.20,8.5,87.0


# Selecting the largest of each group by sorting

In [272]:
movies2 = movies[['Title', 'Year', 'AdjustedForInflationBoxOffice', 'imdbRating', 'Metascore']]
movies2.head()

,Title,Year,AdjustedForInflationBoxOffice,imdbRating,Metascore
2,Star Wars: Episode V - The Empire Strikes Back,1980,13871918.5,8.8,82.0
14,The Dark Knight,2008,613313470.2,9.0,84.0
15,2001: A Space Odyssey,1968,991382.2,8.3,82.0
22,Back to the Future,1985,6846559.2,8.5,87.0
23,The Lord of the Rings: The Fellowship of the Ring,2001,449020000.0,8.8,92.0


In [273]:
# lets sort by year
movies2.sort_values('Year', ascending=False).head()

,Title,Year,AdjustedForInflationBoxOffice,imdbRating,Metascore
452,Indiana Jones and the Kingdom of the Crystal S...,2008,3.676703e+08,6.2,65.0
393,Cloverfield,2008,9.514318e+07,7.0,64.0
405,Iron Man,2008,3.692259e+08,7.9,79.0
372,WALL·E,2008,2.573124e+08,8.4,95.0
14,The Dark Knight,2008,6.133135e+08,9.0,84.0


In [274]:
# sorting by year and imdbRating would be different though...
movies3 = movies2.sort_values(['Year','imdbRating'], ascending=False)
movies3.head()

,Title,Year,AdjustedForInflationBoxOffice,imdbRating,Metascore
14,The Dark Knight,2008,6.133135e+08,9.0,84.0
372,WALL·E,2008,2.573124e+08,8.4,95.0
405,Iron Man,2008,3.692259e+08,7.9,79.0
393,Cloverfield,2008,9.514318e+07,7.0,64.0
452,Indiana Jones and the Kingdom of the Crystal S...,2008,3.676703e+08,6.2,65.0


In [275]:
movie_top_year = movies3.drop_duplicates(subset='Year')
movie_top_year.head()

,Title,Year,AdjustedForInflationBoxOffice,imdbRating,Metascore
14,The Dark Knight,2008,6.133135e+08,9.0,84.0
143,There Will Be Blood,2007,5.016679e+07,8.1,93.0
207,The Departed,2006,1.653750e+08,8.5,85.0
80,Batman Begins,2005,2.653300e+08,8.3,70.0
72,Eternal Sunshine of the Spotless Mind,2004,4.607029e+07,8.3,89.0


In [276]:
# multiple sort values
movies4 = movies[['Title', 'Year', 'Rated', 'AdjustedForInflationBoxOffice']]
movies4_sorted = movies4.sort_values(['Title', 'Year', 'Rated', 'AdjustedForInflationBoxOffice'], 
                                   ascending=[False, False, True, True])
movies4_sorted.drop_duplicates(subset=['Title', 'Year', 'Rated', 'AdjustedForInflationBoxOffice']).head(10)

,Title,Year,Rated,AdjustedForInflationBoxOffice
375,Zodiac,2007,R,4.059000e+07
431,X-Men 2,2003,PG-13,2.942940e+08
372,WALL·E,2008,G,2.573124e+08
417,V for Vendetta,2005,R,9.165000e+07
185,United 93,2006,R,3.937500e+07
451,Unbreakable,2000,PG-13,1.377500e+08
308,Transformers,2007,PG-13,3.860075e+08
143,There Will Be Blood,2007,R,5.016679e+07
171,The Wizard of Oz,1939,PG,7.016963e+07
479,The Son's Room,2001,R,1.054718e+06


# Replicating nlargest with sort_values

In [277]:
# more into that
movies2 = movies[['Title', 'imdbRating', 'AdjustedForInflationBoxOffice']]
movies_smallest_largest = movies2.nlargest(100, 'imdbRating').nsmallest(5, 'AdjustedForInflationBoxOffice')
movies_smallest_largest

,Title,imdbRating,AdjustedForInflationBoxOffice
357,Russian Ark,7.4,52414.60
211,M,8.4,293903.30
119,The Battle of Algiers,8.1,431050.68
255,Port of Shadows,7.7,482037.65
238,Cinema Paradiso,8.5,519997.10


In [278]:
movies2.sort_values('imdbRating', ascending=False).head(100).head()

,Title,imdbRating,AdjustedForInflationBoxOffice
14,The Dark Knight,9.0,613313470.2
33,The Lord of the Rings: The Return of the King,8.9,498680000.0
23,The Lord of the Rings: The Fellowship of the Ring,8.8,449020000.0
239,Forrest Gump,8.8,561000000.0
2,Star Wars: Episode V - The Empire Strikes Back,8.8,13871918.5


In [279]:
movies2.sort_values('imdbRating', ascending=False).head(100).sort_values('AdjustedForInflationBoxOffice').head()

,Title,imdbRating,AdjustedForInflationBoxOffice
357,Russian Ark,7.4,52414.60
211,M,8.4,293903.30
119,The Battle of Algiers,8.1,431050.68
255,Port of Shadows,7.7,482037.65
238,Cinema Paradiso,8.5,519997.10


In [280]:
movies2.nlargest(100, 'imdbRating').tail()

,Title,imdbRating,AdjustedForInflationBoxOffice
416,Lords of Dogtown,7.1,1.431096e+07
393,Cloverfield,7.0,9.514318e+07
404,Dirty Dancing,6.9,4.179133e+06
389,2 Days in Paris,6.8,5.100229e+06
448,Star Wars: Episode I - The Phantom Menace,6.5,6.551200e+08


In [281]:
movies2.sort_values('imdbRating', ascending=False).head(100).tail()

,Title,imdbRating,AdjustedForInflationBoxOffice
308,Transformers,7.1,3.860075e+08
393,Cloverfield,7.0,9.514318e+07
404,Dirty Dancing,6.9,4.179133e+06
389,2 Days in Paris,6.8,5.100229e+06
448,Star Wars: Episode I - The Phantom Menace,6.5,6.551200e+08


# Correlations

In [282]:
movies.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD',
       'BoxOffice', 'Production', 'Website', 'Response', 'Month',
       'AdjustedForInflationBoxOffice'],
      dtype='object')

In [283]:
# no strong correlation between year and BoxOffice
movies["Year"].astype("int64").corr(movies["BoxOffice"])

0.25975908496796857

In [284]:
# critics and imdb users kinda of agree on ratings...
movies["imdbRating"].corr(movies["Metascore"])

0.4448863898133386

In [285]:
# we can see that the older the movie, the better the rating...
# not a very strong correlation, but it is there
movies["Year"].astype("int64").corr(movies["imdbRating"])

-0.18407486714339472

In [286]:
# same thing, even a little stronger, for critics
movies["Year"].astype("int64").corr(movies["Metascore"])

-0.2194801477716047

# Reducing memory by changing data types

In [287]:
movies.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD',
       'BoxOffice', 'Production', 'Website', 'Response', 'Month',
       'AdjustedForInflationBoxOffice'],
      dtype='object')

In [288]:
# lets look at some of the columns and how can we reduce the memory usage
different_cols = ['Title', 'Actors', 'Writer', 'Awards', 'BoxOffice', "AdjustedForInflationBoxOffice", "Rated"]
col2 = movies.loc[:, different_cols]
col2.head()

,Title,Actors,Writer,Awards,BoxOffice,AdjustedForInflationBoxOffice,Rated
2,Star Wars: Episode V - The Empire Strikes Back,"Mark Hamill, Harrison Ford, Carrie Fisher, Bil...","Leigh Brackett (screenplay by), Lawrence Kasda...",Won 1 Oscar. Another 21 wins & 19 nominations.,4548170.0,13871918.5,PG
14,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","Jonathan Nolan (screenplay), Christopher Nolan...",Won 2 Oscars. Another 152 wins & 155 nominations.,533316061.0,613313470.2,PG-13
15,2001: A Space Odyssey,"Keir Dullea, Gary Lockwood, William Sylvester,...","Stanley Kubrick (screenplay), Arthur C. Clarke...",Won 1 Oscar. Another 13 wins & 10 nominations.,135620.0,991382.2,G
22,Back to the Future,"Michael J. Fox, Christopher Lloyd, Lea Thompso...","Robert Zemeckis, Bob Gale",Won 1 Oscar. Another 19 wins & 25 nominations.,2925880.0,6846559.2,PG
23,The Lord of the Rings: The Fellowship of the Ring,"Alan Howard, Noel Appleby, Sean Astin, Sala Baker","J.R.R. Tolkien (novel), Fran Walsh (screenplay...",Won 4 Oscars. Another 113 wins & 123 nominations.,314000000.0,449020000.0,PG-13


In [289]:
col2.dtypes

Title                             object
Actors                            object
Writer                            object
Awards                            object
BoxOffice                        float64
AdjustedForInflationBoxOffice    float64
Rated                             object
dtype: object

In [290]:
#Find the memory usage of each column with the memory_usage method:
original_mem = col2.memory_usage(deep=True)
original_mem

Index                              824
Title                             7690
Actors                           12113
Writer                           14797
Awards                           10294
BoxOffice                          824
AdjustedForInflationBoxOffice      824
Rated                             6601
dtype: int64

In [291]:
# I do not see any reaon to change the data types - will skip this.

In [292]:
# We have a lot of unique values - excepted for the Rated column
col2.select_dtypes(include=['object']).nunique()

Title     103
Actors    103
Writer    102
Awards    102
Rated       6
dtype: int64

In [293]:
# we can convert the Rated column into a categorical data type
# to save memory space
col2['Rated'] = col2['Rated'].astype('category')
col2.dtypes

Title                              object
Actors                             object
Writer                             object
Awards                             object
BoxOffice                         float64
AdjustedForInflationBoxOffice     float64
Rated                            category
dtype: object

In [294]:
# Let compute the memory again
# We will see a difference in the rated column
new_mem = col2.memory_usage(deep=True)
new_mem

Index                              824
Title                             7690
Actors                           12666
Writer                           15157
Awards                           10294
BoxOffice                          824
AdjustedForInflationBoxOffice      824
Rated                              647
dtype: int64

In [295]:
# percentage of memory saved for each type
new_mem / original_mem
# we can see we saved 90% of memory that was used by Rated

Index                            1.000000
Title                            1.000000
Actors                           1.045653
Writer                           1.024329
Awards                           1.000000
BoxOffice                        1.000000
AdjustedForInflationBoxOffice    1.000000
Rated                            0.098015
dtype: float64

## There's more...

In [296]:
movies.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD',
       'BoxOffice', 'Production', 'Website', 'Response', 'Month',
       'AdjustedForInflationBoxOffice'],
      dtype='object')

In [297]:
movies[['Language', 'Country']].memory_usage(deep=True)

Index        824
Language    7513
Country     6792
dtype: int64

In [298]:
# more into memory
# changing a single value would lead to the change of the memory usage
movies.loc[0, 'Language'] = 10000000
movies.loc[0, 'Country'] = str(movies.loc[0, 'Country']) + str('a')
# movies.loc[1, 'INSTNM'] = movies.loc[1, 'INSTNM'] + 'a'
movies[['Language', 'Country']].memory_usage(deep=True)

Index       5952
Language    7549
Country     6873
dtype: int64

In [299]:
movies['Language'].dtype

dtype('O')

In [300]:
movies['Language'].astype('int8') # ValueError

ValueError: invalid literal for int() with base 10: 'English'

In [301]:
# more statistical, numerical, descriptions
movies.describe(include=['int64', 'float64']).T

,count,mean,std,min,25%,50%,75%,max
Year,103.0,1.998417e+03,1.586000e+01,1925.0,2000.50,2004.0,2006.0,2008.0
Metascore,99.0,7.753535e+01,1.197303e+01,46.0,68.00,80.0,85.5,100.0
imdbRating,103.0,7.828155e+00,5.645435e-01,6.1,7.50,7.9,8.2,9.0
imdbVotes,103.0,4.686225e+05,3.679558e+05,6831.0,216119.50,391323.0,632194.5,1969949.0
BoxOffice,103.0,9.913201e+07,1.267996e+08,17599.0,5166076.00,44566004.0,155194037.0,533316061.0
AdjustedForInflationBoxOffice,103.0,1.332490e+08,1.694818e+08,52414.6,8749140.73,62860656.6,190408024.0,655120000.0


In [302]:
movies.describe(include=[np.int64, np.float64]).T

,count,mean,std,min,25%,50%,75%,max
Year,103.0,1.998417e+03,1.586000e+01,1925.0,2000.50,2004.0,2006.0,2008.0
Metascore,99.0,7.753535e+01,1.197303e+01,46.0,68.00,80.0,85.5,100.0
imdbRating,103.0,7.828155e+00,5.645435e-01,6.1,7.50,7.9,8.2,9.0
imdbVotes,103.0,4.686225e+05,3.679558e+05,6831.0,216119.50,391323.0,632194.5,1969949.0
BoxOffice,103.0,9.913201e+07,1.267996e+08,17599.0,5166076.00,44566004.0,155194037.0,533316061.0
AdjustedForInflationBoxOffice,103.0,1.332490e+08,1.694818e+08,52414.6,8749140.73,62860656.6,190408024.0,655120000.0


In [303]:
# another error - value conversion error
movies['Rated'] = movies['RELAFFIL'].astype(np.int8)

KeyError: 'RELAFFIL'

In [304]:
movies.describe(include=['int', 'float']).T  # defaults to 64 bit int/floats

,count,mean,std,min,25%,50%,75%,max
Year,103.0,1.998417e+03,1.586000e+01,1925.0,2000.50,2004.0,2006.0,2008.0
Metascore,99.0,7.753535e+01,1.197303e+01,46.0,68.00,80.0,85.5,100.0
imdbRating,103.0,7.828155e+00,5.645435e-01,6.1,7.50,7.9,8.2,9.0
imdbVotes,103.0,4.686225e+05,3.679558e+05,6831.0,216119.50,391323.0,632194.5,1969949.0
BoxOffice,103.0,9.913201e+07,1.267996e+08,17599.0,5166076.00,44566004.0,155194037.0,533316061.0
AdjustedForInflationBoxOffice,103.0,1.332490e+08,1.694818e+08,52414.6,8749140.73,62860656.6,190408024.0,655120000.0


In [305]:
movies.describe(include=['number']).T  # also works as the default int/float are 64 bits

,count,mean,std,min,25%,50%,75%,max
Year,103.0,1.998417e+03,1.586000e+01,1925.0,2000.50,2004.0,2006.0,2008.0
Metascore,99.0,7.753535e+01,1.197303e+01,46.0,68.00,80.0,85.5,100.0
imdbRating,103.0,7.828155e+00,5.645435e-01,6.1,7.50,7.9,8.2,9.0
imdbVotes,103.0,4.686225e+05,3.679558e+05,6831.0,216119.50,391323.0,632194.5,1969949.0
BoxOffice,103.0,9.913201e+07,1.267996e+08,17599.0,5166076.00,44566004.0,155194037.0,533316061.0
AdjustedForInflationBoxOffice,103.0,1.332490e+08,1.694818e+08,52414.6,8749140.73,62860656.6,190408024.0,655120000.0


In [306]:
# memory usage
movies.index = pd.Int64Index(movies.index)
movies.index.memory_usage()

832

# Interesting Things I Found After Finishing the Assignment


* Christopher Nolan is the highest frequent director for the top 500 movies!
* There are 2 beauty and the beast movies
* The Godfather is the highest rated movie with 9.2 imdbRating and a 100 for the Metascore.
* Metascore is the rating of critics while IMDB rating is for users on the website. 
* There is a correlation of 0.42, which is moderate, between Metascore and IMDB rating.
* The older the movie, the higher the rating, can be seen as there is a negative correlation between Metascore/imdbRating and Year. This is stronger for critics than IMDB users.
